# Modeling Crop Yield: Landsat + Sentinel
## Python modules

In [1]:
%load_ext lab_black

In [2]:
import os
import glob
from pyhere import here
import random

import itertools

from task_modeling_utils import *

In [231]:
oos = pd.read_csv(
    here(
        "data",
        "results",
        "2_sensor_oos_predictions_n-splits-10_2023-05-22_rcf_climate.csv",
    )
)
oos = oos.loc[
    (oos.variables == "ndvi") & (oos.split == 0) & (oos.random_state == 670487)
]
oos

,data_fold,district,year,yield_mt,log_yield,oos_prediction,val_fold,demean_log_yield,demean_oos_prediction,split,random_state,variables
0,train,Kaoma,2019,0.523864,0.182946,0.279325,1,-0.183223,-0.090748,0,670487,ndvi
1,train,Kaoma,2017,1.540420,0.404906,0.368853,1,0.038736,-0.001220,0,670487,ndvi
2,train,Sinazongwe,2020,0.589120,0.201157,0.205528,1,-0.003610,-0.026678,0,670487,ndvi
3,train,Kasempa,2020,3.349064,0.638396,0.555924,1,0.077697,0.004680,0,670487,ndvi
4,train,Shangombo,2020,0.741213,0.240852,0.166590,1,0.107542,0.011047,0,670487,ndvi
...,...,...,...,...,...,...,...,...,...,...,...,...
415,test,Siavonga,2016,1.186227,0.339695,0.228925,6,0.100687,0.064225,0,670487,ndvi
416,test,Sinazongwe,2017,1.041549,0.309960,0.314992,6,0.000000,0.000000,0,670487,ndvi
417,test,Siavonga,2018,0.506208,0.177885,0.146035,6,-0.061123,-0.018665,0,670487,ndvi
418,test,Mbala,2019,1.830338,0.451838,0.476251,6,-0.039755,-0.011767,0,670487,ndvi


In [232]:
group = ["district"]
# group = ["district", "data_fold"]
# group = ["district", "val_fold"]
# group = ["district", "val_fold", "split", "random_state"]
oos["demean_log_yield"] = oos["log_yield"] - oos.groupby(group)["log_yield"].transform(
    "mean"
)
# group.append("data_fold")
oos["demean_oos_prediction"] = oos["oos_prediction"] - oos.groupby(group)[
    "log_yield"
].transform("mean")
oos

,data_fold,district,year,yield_mt,log_yield,oos_prediction,val_fold,demean_log_yield,demean_oos_prediction,split,random_state,variables
0,train,Kaoma,2019,0.523864,0.182946,0.279325,1,-0.192782,-0.096403,0,670487,ndvi
1,train,Kaoma,2017,1.540420,0.404906,0.368853,1,0.029177,-0.006875,0,670487,ndvi
2,train,Sinazongwe,2020,0.589120,0.201157,0.205528,1,-0.021142,-0.016772,0,670487,ndvi
3,train,Kasempa,2020,3.349064,0.638396,0.555924,1,0.077697,-0.004775,0,670487,ndvi
4,train,Shangombo,2020,0.741213,0.240852,0.166590,1,0.071072,-0.003190,0,670487,ndvi
...,...,...,...,...,...,...,...,...,...,...,...,...
415,test,Siavonga,2016,1.186227,0.339695,0.228925,6,0.111745,0.000975,0,670487,ndvi
416,test,Sinazongwe,2017,1.041549,0.309960,0.314992,6,0.087661,0.092693,0,670487,ndvi
417,test,Siavonga,2018,0.506208,0.177885,0.146035,6,-0.050065,-0.081915,0,670487,ndvi
418,test,Mbala,2019,1.830338,0.451838,0.476251,6,-0.072793,-0.048381,0,670487,ndvi


In [233]:
test = oos.copy()
test = test[test.data_fold == "test"]
train = oos.copy()
train = train[train.data_fold == "train"]

In [234]:
r2_score(train.demean_log_yield, train.demean_oos_prediction), r2_score(
    test.demean_log_yield, test.demean_oos_prediction
)

(0.19985888153839537, 0.3320352314844004)

[pod-login1.podcluster:67718] tcp_peer_recv_connect_ack: invalid header type: 114
[pod-login1.podcluster:67718] tcp_peer_recv_connect_ack: invalid header type: 24
[pod-login1.podcluster:67718] tcp_peer_recv_connect_ack: invalid header type: 226
[pod-login1.podcluster:67718] tcp_peer_recv_connect_ack: invalid header type: 194
[pod-login1.podcluster:67718] tcp_peer_recv_connect_ack: invalid header type: 133
[pod-login1.podcluster:67718] tcp_peer_recv_connect_ack: invalid header type: 16
[pod-login1.podcluster:67718] tcp_peer_recv_connect_ack: invalid header type: 73
[pod-login1.podcluster:67718] tcp_peer_recv_connect_ack: invalid header type: 84
[pod-login1.podcluster:67718] tcp_peer_recv_connect_ack: invalid header type: 128


In [136]:
r2_score(test.log_yield, test.oos_prediction)

0.8086417044621875

In [170]:
results = pd.read_csv(
    here("data", "results", "2_sensor_top-mod_n-splits-10_2023-05-22_rcf_climate.csv")
)
results = results[results.variables == "ndvi"]
np.mean(results.val_R2)
results

,split,random_state,variables,country,year_range,satellite_1,bands_1,num_features_1,points_1,month_range_1,...,train_r2,test_R2,test_r,test_r2,demean_cv_R2,demean_cv_r,demean_cv_r2,demean_test_R2,demean_test_r,demean_test_r2
0,0,670487,ndvi,ZMB,2016-2021,landsat-c2-l2,r-g-b-nir-swir16-swir22,1024,20,4-9,...,0.967818,0.847675,0.938276,0.880362,0.209737,0.523837,0.274405,0.545070,0.738365,0.545183
1,1,116739,ndvi,ZMB,2016-2021,landsat-c2-l2,r-g-b-nir-swir16-swir22,1024,20,4-9,...,0.965704,0.803469,0.900288,0.810518,0.131235,0.467381,0.218445,0.657664,0.811210,0.658062
2,2,26225,ndvi,ZMB,2016-2021,landsat-c2-l2,r-g-b-nir-swir16-swir22,1024,20,4-9,...,0.957807,0.824435,0.908954,0.826197,0.144580,0.461305,0.212803,0.700661,0.860644,0.740707
3,3,777572,ndvi,ZMB,2016-2021,landsat-c2-l2,r-g-b-nir-swir16-swir22,1024,20,4-9,...,0.964629,0.863111,0.929479,0.863932,0.204904,0.523020,0.273550,0.478438,0.697391,0.486354
4,4,288389,ndvi,ZMB,2016-2021,landsat-c2-l2,r-g-b-nir-swir16-swir22,1024,20,4-9,...,0.969930,0.814105,0.908593,0.825542,0.309138,0.577654,0.333684,0.428592,0.690432,0.476696
5,5,256787,ndvi,ZMB,2016-2021,landsat-c2-l2,r-g-b-nir-swir16-swir22,1024,20,4-9,...,0.988286,0.780335,0.884826,0.782917,0.086451,0.491176,0.241254,0.212076,0.469716,0.220633
6,6,234053,ndvi,ZMB,2016-2021,landsat-c2-l2,r-g-b-nir-swir16-swir22,1024,20,4-9,...,0.969840,0.839898,0.919857,0.846137,0.281425,0.573529,0.328936,0.519446,0.742793,0.551741
7,7,146316,ndvi,ZMB,2016-2021,landsat-c2-l2,r-g-b-nir-swir16-swir22,1024,20,4-9,...,0.958149,0.800192,0.895981,0.802781,0.260074,0.542605,0.294420,0.049373,0.348908,0.121737
8,8,772246,ndvi,ZMB,2016-2021,landsat-c2-l2,r-g-b-nir-swir16-swir22,1024,20,4-9,...,0.963923,0.843533,0.927076,0.859469,0.168657,0.495024,0.245049,0.477862,0.691418,0.478059
9,9,107473,ndvi,ZMB,2016-2021,landsat-c2-l2,r-g-b-nir-swir16-swir22,1024,20,4-9,...,0.964986,0.868590,0.932461,0.869483,0.165132,0.484519,0.234759,0.606238,0.793241,0.629231


In [79]:
file_pattern = str(here("data", "results", "2_sensor_top-mod_n-splits-10_*_*.csv"))
files = glob.glob(pathname=file_pattern)
files

['/home/cmolitor/crop-modeling/code/3_task_modeling/../../data/results/2_sensor_top-mod_n-splits-10_2023-05-11_4.csv',
 '/home/cmolitor/crop-modeling/code/3_task_modeling/../../data/results/2_sensor_top-mod_n-splits-10_2023-05-12_15.csv',
 '/home/cmolitor/crop-modeling/code/3_task_modeling/../../data/results/2_sensor_top-mod_n-splits-10_2023-05-11_3.csv',
 '/home/cmolitor/crop-modeling/code/3_task_modeling/../../data/results/2_sensor_top-mod_n-splits-10_2023-05-12_9.csv',
 '/home/cmolitor/crop-modeling/code/3_task_modeling/../../data/results/2_sensor_top-mod_n-splits-10_2023-05-12_17.csv',
 '/home/cmolitor/crop-modeling/code/3_task_modeling/../../data/results/2_sensor_top-mod_n-splits-10_2023-05-11_6.csv',
 '/home/cmolitor/crop-modeling/code/3_task_modeling/../../data/results/2_sensor_top-mod_n-splits-10_2023-05-13_18.csv',
 '/home/cmolitor/crop-modeling/code/3_task_modeling/../../data/results/2_sensor_top-mod_n-splits-10_2023-05-11_1.csv',
 '/home/cmolitor/crop-modeling/code/3_task_mo

In [54]:
file_pattern = str(here("data", "results", "2_sensor_top-mod_n-splits-10_*_*.csv"))
files = glob.glob(pathname=file_pattern)
results = merge_files(files)
results

,split,random_state,country,year_range,satellite_1,bands_1,num_features_1,points_1,month_range_1,limit_months_1,...,test_R2,test_r,test_r2,demean_cv_R2,demean_cv_r,demean_cv_r2,demean_test_R2,demean_test_r,demean_test_r2,variables
0,0,670487,ZMB,2013-2021,landsat-8-c2-l2,1-2-3-4-5-6-7,1000,15,4-9,True,...,0.811017,0.900635,0.811144,0.026742,0.355246,0.126199,0.275154,0.525508,0.276159,NaN
1,1,116739,ZMB,2013-2021,landsat-8-c2-l2,1-2-3-4-5-6-7,1000,15,4-9,True,...,0.825590,0.909478,0.827150,-0.022876,0.331399,0.109825,0.345143,0.599142,0.358971,NaN
2,2,26225,ZMB,2013-2021,landsat-8-c2-l2,1-2-3-4-5-6-7,1000,15,4-9,True,...,0.823030,0.910208,0.828478,-0.038391,0.319619,0.102157,0.289007,0.539315,0.290860,NaN
3,3,777572,ZMB,2013-2021,landsat-8-c2-l2,1-2-3-4-5-6-7,1000,15,4-9,True,...,0.757509,0.872604,0.761437,0.034316,0.373193,0.139273,0.115665,0.445977,0.198896,NaN
4,4,288389,ZMB,2013-2021,landsat-8-c2-l2,1-2-3-4-5-6-7,1000,15,4-9,True,...,0.827615,0.910341,0.828722,-0.087215,0.270659,0.073256,0.392660,0.633720,0.401601,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1155,5,256787,ZMB,2016-2021,landsat-c2-l2,r-g-b-nir-swir16-swir22,1024,20,4-9,True,...,0.804027,0.900171,0.810308,0.151647,0.455496,0.207476,0.179910,0.444252,0.197359,NaN
1156,6,234053,ZMB,2016-2021,landsat-c2-l2,r-g-b-nir-swir16-swir22,1024,20,4-9,True,...,0.837362,0.916496,0.839965,0.033580,0.364469,0.132838,0.648047,0.824200,0.679306,NaN
1157,7,146316,ZMB,2016-2021,landsat-c2-l2,r-g-b-nir-swir16-swir22,1024,20,4-9,True,...,0.718199,0.847727,0.718641,0.051969,0.469034,0.219993,0.154963,0.443899,0.197047,NaN
1158,8,772246,ZMB,2016-2021,landsat-c2-l2,r-g-b-nir-swir16-swir22,1024,20,4-9,True,...,0.845678,0.922269,0.850581,0.201498,0.502066,0.252070,0.505067,0.736965,0.543117,NaN


In [6]:
results.columns

Index(['split', 'random_state', 'country', 'year_range', 'satellite_1',
       'bands_1', 'num_features_1', 'points_1', 'month_range_1',
       'limit_months_1', 'crop_mask_1', 'weighted_avg_1', 'satellite_2',
       'bands_2', 'num_features_2', 'points_2', 'month_range_2',
       'limit_months_2', 'crop_mask_2', 'weighted_avg_2', 'hot_encode',
       'total_n', 'train_n', 'test_n', 'best_reg_param', 'mean_of_val_R2',
       'val_R2', 'val_r', 'val_r2', 'train_R2', 'train_r', 'train_r2',
       'test_R2', 'test_r', 'test_r2', 'demean_cv_R2', 'demean_cv_r',
       'demean_cv_r2', 'demean_test_R2', 'demean_test_r', 'demean_test_r2'],
      dtype='object')

In [7]:
max(results.val_R2)

0.8390845735029497

In [8]:
groupby_cols = [
    "country",
    "year_range",
    "satellite_1",
    "bands_1",
    "num_features_1",
    "points_1",
    "month_range_1",
    "limit_months_1",
    "crop_mask_1",
    "weighted_avg_1",
    "satellite_2",
    "bands_2",
    "num_features_2",
    "points_2",
    "month_range_2",
    "limit_months_2",
    "crop_mask_2",
    "weighted_avg_2",
    "hot_encode",
]

In [9]:
results_summary = results.groupby(groupby_cols, as_index=False).agg(
    {
        "val_R2": "mean",
        "test_R2": "mean",
        "demean_cv_R2": "mean",
        "demean_cv_r2": "mean",
        "demean_test_R2": "mean",
    }
)
results_summary = results_summary.sort_values("val_R2", ascending=False)  # .head(20)
results_summary.iloc[0:1, :]

,country,year_range,satellite_1,bands_1,num_features_1,points_1,month_range_1,limit_months_1,crop_mask_1,weighted_avg_1,...,month_range_2,limit_months_2,crop_mask_2,weighted_avg_2,hot_encode,val_R2,test_R2,demean_cv_R2,demean_cv_r2,demean_test_R2
76,ZMB,2016-2021,landsat-c2-l2,r-g-b-nir-swir16-swir22,1024,20,4-9,True,True,False,...,1-12,False,True,False,True,0.80024,0.825819,0.169455,0.245413,0.452211


In [29]:
f1 = "landsat-c2-l2_bands-r-g-b-nir-swir16-swir22_ZMB_20k-points_1024-features_yr-2009-2021_mn-4-9_lm-True_cm-True_wa-False_summary.feather"
f2 = "sentinel-2-l2a_bands-2-3-4-8_ZMB_15k-points_1000-features_yr-2016-2022_mn-1-12_lm-False_cm-True_wa-False_summary.feather"

In [32]:
# f1 = "landsat-c2-l2_bands-r-g-b-nir-swir16-swir22_ZMB_20k-points_1024-features_yr-2009-2021_mn-4-9_lm-True_cm-True_wa-True_summary.feather"
pd.read_feather(here("data", "random_features", "summary", f2))

,district,year,0_1,0_2,0_3,0_4,0_5,0_6,0_7,0_8,...,999_5,999_6,999_7,999_8,999_9,999_10,999_11,999_12,crop_perc,yield_mt
0,Chadiza,2016,0.0,0.0,0.0,0.000000e+00,8.029142e-07,0.0,0.0,0.000002,...,3.567133,3.853844,3.958063,4.344390,4.379719,4.411875,4.560238,3.610760,0.351285,1.876427
1,Chadiza,2017,0.0,0.0,0.0,7.159268e-07,3.442723e-07,0.0,0.0,0.000002,...,3.119058,3.424247,3.588746,4.128900,4.055817,4.391447,4.598734,3.215563,0.351285,2.882947
2,Chadiza,2018,0.0,0.0,0.0,1.419619e-07,1.716714e-07,0.0,0.0,0.000000,...,3.067993,3.246320,3.620674,4.081536,4.352846,3.996915,4.333624,3.868718,0.351285,1.299279
3,Chadiza,2019,0.0,0.0,0.0,3.004383e-06,0.000000e+00,0.0,0.0,0.000000,...,3.407543,3.597741,3.933495,4.052879,4.327971,4.825222,4.669475,3.610760,0.351285,2.131008
4,Chadiza,2020,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.000000,...,3.516882,3.679166,3.902666,4.278298,4.543046,4.406098,4.560024,3.691440,0.351285,2.626538
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
485,Zambezi,2018,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.000000,...,2.675449,2.826331,3.069272,3.217198,3.494478,3.610686,3.563169,3.425959,0.037132,1.622273
486,Zambezi,2019,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.000000,...,2.807577,3.079239,3.304896,3.623420,3.590696,3.633996,3.412879,3.425959,0.037132,1.184923
487,Zambezi,2020,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.000000,...,2.653428,2.936435,3.054644,3.350338,3.543792,3.930742,3.531136,3.142700,0.037132,1.689628
488,Zambezi,2021,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.000000,...,2.754887,2.950750,3.153819,3.299913,3.565012,3.661414,3.528146,3.425959,0.037132,2.981741


In [128]:
# Generate n random seeds
n_splits = 10
random_seeds = [random.randint(0, 1_000_000) for _ in range(n_splits)]

directory = here("data", "random_features", "summary")
files = [
    f for f in os.listdir(directory) if f not in (".gitkeep", ".ipynb_checkpoints")
]
files = [f for f in files if not (f.startswith("landsat-8") and "lm-False" in f)]
files = [f for f in files if not (f.startswith("sentinel") and "lm-True" in f)]
files = [f for f in files if "cm-True" in f]
# files = [f for f in files if "wa-False" in f]

combinations = list(itertools.combinations(files, 2))
combinations = [
    t for t in combinations if not ("landsat-c2" in t[0] and "landsat-c2" in t[1])
]

kwarg_list = [
    {
        "f1": f1,
        "f2": f2,
        "he": False,
        "split": split,
        "random_state": random_state,
        "include_climate": False,
        "variable_groups": None,
        "n_splits": 5,
    }
    for f1, f2 in combinations
    for split, random_state in enumerate(random_seeds)
]

chunked_kwarg_list = list(chunks(kwarg_list, 60))

In [135]:
for i, chunk in enumerate(chunked_kwarg_list[9:10]):
    print(i)

0


In [134]:
chunked_kwarg_list[12:]

[[{'f1': 'landsat-c2-l2_bands-r-g-b-nir-swir16-swir22_ZMB_20k-points_1024-features_yr-2009-2021_mn-4-9_lm-True_cm-True_wa-False_summary.feather',
   'f2': 'sentinel-2-l2a_bands-2-3-4_ZMB_15k-points_1000-features_yr-2016-2022_mn-1-12_lm-False_cm-True_wa-False_summary.feather',
   'he': False,
   'split': 0,
   'random_state': 675675,
   'include_climate': False,
   'variable_groups': None,
   'n_splits': 5},
  {'f1': 'landsat-c2-l2_bands-r-g-b-nir-swir16-swir22_ZMB_20k-points_1024-features_yr-2009-2021_mn-4-9_lm-True_cm-True_wa-False_summary.feather',
   'f2': 'sentinel-2-l2a_bands-2-3-4_ZMB_15k-points_1000-features_yr-2016-2022_mn-1-12_lm-False_cm-True_wa-False_summary.feather',
   'he': False,
   'split': 1,
   'random_state': 32297,
   'include_climate': False,
   'variable_groups': None,
   'n_splits': 5},
  {'f1': 'landsat-c2-l2_bands-r-g-b-nir-swir16-swir22_ZMB_20k-points_1024-features_yr-2009-2021_mn-4-9_lm-True_cm-True_wa-False_summary.feather',
   'f2': 'sentinel-2-l2a_bands-2-

In [ ]:
file_pattern = str(here("data", "results", "2_sensor_results_*_*.csv"))
files = glob.glob(pathname=file_pattern)
results = merge_files(files)
sorted(files)

In [ ]:
# top = results.test_R2.sort_values().index[-1]
# results.iloc[top:top+1, 1:20]

top = results.val_R2.sort_values().index[-1]
results.iloc[top : top + 1, 10:]
# results.iloc[top : top + 1, 1:20]

In [ ]:
f1 = "landsat-c2-l2_bands-r-g-b-nir-swir16-swir22_ZMB_20k-points_1024-features_yr-2009-2021_mn-1-12_lm-False_cm-True_wa-False_summary.feather"
f2 = "sentinel-2-l2a_bands-2-3-4-8_ZMB_15k-points_1000-features_yr-2016-2022_mn-1-12_lm-False_cm-True_wa-False_summary.feather"

In [ ]:
random.seed(42)

# Define the number of stratified random splits to perform
n_splits = 100  # Generate n random seeds
random_seeds = [random.randint(0, 1_000_000) for _ in range(n_splits)]

paramlist = [
    (
        f1,
        f2,
        "True",
        split,
        random_state,
    )
    for split, random_state in enumerate(random_seeds)
]

In [11]:
%%time
## TESTING  
f1 = "landsat-c2-l2_bands-r-g-b-nir-swir16-swir22_ZMB_20k-points_1024-features_yr-2009-2021_mn-4-9_lm-True_cm-True_wa-False_summary.feather"
f2 = "sentinel-2-l2a_bands-2-3-4-8_ZMB_15k-points_1000-features_yr-2016-2022_mn-1-12_lm-False_cm-True_wa-False_summary.feather"
he = True
split=0
random_state=670487
n_splits = 5
include_climate = True
# variable_groups = ['tmp', 'ndvi', 'pre']
# variable_groups = ['tmp', 'ndvi']
variable_groups = ['ndvi']
# variable_groups = None 

return_oos_predictions = True

#########################################     SET PARAMS    #########################################    
satellite1, bands1, country_code, points1, yrs1, mns1,\
num_features1, limit_months1, crop_mask1, weighted_avg1 = split_fn(f1)

satellite2, bands2, country_code, points2, yrs2, mns2,\
num_features2, limit_months2, crop_mask2, weighted_avg2 = split_fn(f2)

if variable_groups is None:
    variable_groups_str = "None"
else:
    variable_groups_str = "_".join(variable_groups)

print(f"""Begin with paramters:
F1: {f1}
F2: {f2}
One-hot encoding: {he}
Split: {split}
Random state: {random_state}
N-splits: {n_splits}
Include climate: {include_climate}
Climate vars: {variable_groups_str}
""", flush=True)

#########################################     READ DATA    #########################################
features_1 = pd.read_feather(here('data', 'random_features', 'summary', f1))
features_2 = pd.read_feather(here('data', 'random_features', 'summary', f2))
if include_climate:
    climate_df = pd.read_csv(here('data', 'climate', 'climate_summary.csv'))

#########################################     CLEAN DATA    #########################################  
min_year = max(min(features_1.year), min(features_2.year))
max_year = min(max(features_1.year), max(features_2.year))

features_1 = features_1[features_1.year >= min_year]
features_2 = features_2[features_2.year >= min_year]

features_1 = features_1[features_1.year <= max_year]
features_2 = features_2[features_2.year <= max_year]

features_1.drop(['crop_perc'], axis=1, errors='ignore', inplace=True)
features_2.drop(['crop_perc'], axis=1, errors='ignore', inplace=True)

#########################################     JOIN FEATURES    #########################################  
drop_cols = ['district', 'year', 'yield_mt']

features_1 = features_1.set_index(drop_cols).add_prefix("f1_")
features_2 = features_2.set_index(drop_cols).add_prefix("f2_")

features = features_1.join(features_2).reset_index()
features = features[~features.isna().any(axis = 1)]

#########################################    JOIN CLIMATE VARS    #########################################
if include_climate:
    keep_cols = []

    for var in variable_groups:
        tmp = climate_df.columns[climate_df.columns.to_series().str.contains(var)].tolist()
        keep_cols.append(tmp)

    keep_cols = [*drop_cols, *[col for cols in keep_cols for col in cols]]

    climate_df = climate_df.loc[:, keep_cols]

    features = (
        features.set_index(drop_cols).join(climate_df.set_index(drop_cols)).reset_index()
    )
    features = features[features.year <= max(climate_df.year)]

#########################################    STANDARDIZE FEATURES    #########################################
features = features.set_index(drop_cols)
features_scaled = StandardScaler().fit_transform(features.values)
features = pd.DataFrame(features_scaled, index=features.index).reset_index()
features.columns = features.columns.astype(str)

#########################################     CLEAN AND COPY    #########################################
yrs = f"{min(features.year)}-{max(features.year)}"
n_fts_1 = features_1.shape[1]
n_fts_2 = features_2.shape[1]
n_districts = len(features.district.unique())

if include_climate:
    n_climate_cols = climate_df.shape[1] - len(drop_cols)

    i = 0
    n_climate_groups = []
    for cols in range(n_climate_cols):
        if cols % 12 == 0:
            i += 1
            n_climate_groups.append(i)
    n_climate_groups

crop_yield = features.copy().loc[:, tuple(drop_cols)]
crop_yield["log_yield"] = np.log10(crop_yield.yield_mt.to_numpy() + 1)

del features_1, features_2
gc.collect()

#########################################    HOT ENCODE    #########################################
if he:
    drop_cols.remove("district")
    features = pd.get_dummies(features, columns=["district"], drop_first=False)
else:
    pass

#########################################     K-FOLD SPLIT    #########################################
x_all = features.drop(drop_cols, axis=1)
y_all = np.log10(features.yield_mt.to_numpy() + 1)
x_train, x_test, y_train, y_test = train_test_split(
    x_all, y_all, test_size=0.2, random_state=random_state
)

del features
gc.collect()

#########################################     K-FOLD CV    ###########################################
### SETUP
tic = time.time()
kfold = KFold(n_splits=n_splits)
alphas = {"alpha": np.logspace(-1, 1, base=10, num=3)}

### LAMBDA INDICIES
start = [0, n_fts_1]
end = [n_fts_1, x_train.shape[1]]

if include_climate:
    start.append(n_fts_1 + n_fts_2)  
    end.append(n_fts_1 + n_fts_2)  

    for n in n_climate_groups:
        x = n * 12
        y = n_fts_1 + n_fts_2 + x
        start.append(y)
        end.append(y)

if not include_climate and he:
    start.append(x_train.shape[1] - n_districts)
    end.append(x_train.shape[1] - n_districts)

end.sort()

print(f'Group indicies {start}\n\t\t  {end}', end='\n\n')

### GRID SEARCH - FINDING BEST REGULARIZATION PARAMETER(S)
best_lambdas, best_scores, best_model = kfold_rr_multi_lambda_tuning(
    X=x_train,
    y=y_train, 
    grid=alphas.get('alpha'), 
    n_splits=n_splits,
    start=start,
    end=end, 
    static_lam=1,
    verbose=2,
    show_linalg_warning=False,
    fit_model_after_tuning=True
)
### PREDICT WITH BEST HYPERPARAMETER(S)
val_predictions   = cross_val_predict(best_model, X=x_train, y=y_train, cv=kfold)   
train_predictions = best_model.predict(x_train)
test_predictions  = best_model.predict(x_test)
print(f"""
Finish:
F1: {f1}
F2: {f2}
One-hot encoding: {he}
Split: {split}
Random state: {random_state}
N-splits: {n_splits}
Include climate: {include_climate}
Climate vars: {variable_groups_str}
Final Val R2:  {r2_score(y_train, val_predictions):0.4f} 
Final Test R2: {r2_score(y_test, test_predictions):0.4f}
Total time: {(time.time()-tic)/60:0.2f} minutes
""", flush=True)

#########################################     DE-MEAN TRAIN R2    #########################################
fold_list = []
for i in range(n_splits):
    idx = len(list(kfold.split(y_train))[i][1])
    fold = np.repeat(i + 1, idx).tolist()
    fold_list.append(fold)
fold_list = [item for sublist in fold_list for item in sublist]

train_split = pd.DataFrame(
    np.repeat("train", len(x_train)), columns=["data_fold"], index=x_train.index
)
train_split = train_split.join(
    crop_yield.copy()[crop_yield.index.isin(x_train.index)]
)
train_split["oos_prediction"] = np.maximum(val_predictions, 0)
train_split["val_fold"] = fold_list
train_split = demean_by_group(train_split, predicted="oos_prediction", group=["district"])

#########################################     DE-MEAN TEST R2    #########################################
test_split = pd.DataFrame({"data_fold": np.repeat("test", len(x_test))}, index=x_test.index)
test_split = test_split.join(crop_yield.copy()[crop_yield.index.isin(x_test.index)])
test_split["oos_prediction"] = np.maximum(best_model.predict(x_test), 0)
test_split["val_fold"] = n_splits + 1
test_split = demean_by_group(test_split, predicted="oos_prediction", group=["district"])

#########################################     OUT OF SAMPLE PREDICTIONS    #########################################
oos_preds = pd.concat([train_split, test_split])
oos_preds[["split", "random_state"]] = split, random_state
oos_preds["variables"] = variable_groups_str

#########################################     SAVE RESULTS    #########################################
d = {
    "split": split,
    "random_state": random_state,
    "variables": variable_groups_str,
    "country": country_code[0],
    "year_range": yrs,
    "satellite_1": satellite1[0],
    "bands_1": bands1,
    "num_features_1": num_features1,
    "points_1": points1,
    "month_range_1": mns1,
    "limit_months_1": limit_months1,
    "crop_mask_1": crop_mask1,
    "weighted_avg_1": weighted_avg1,
    "satellite_2": satellite2[0],
    "bands_2": bands2,
    "num_features_2": num_features2,
    "points_2": points2,
    "month_range_2": mns2,
    "limit_months_2": limit_months2,
    "crop_mask_2": crop_mask2,
    "weighted_avg_2": weighted_avg2,
    "hot_encode": he,
    "total_n": len(x_all),
    "train_n": len(x_train),
    "test_n": len(x_test),
    "best_reg_param": [best_lambdas],
    "mean_of_val_R2": [best_scores],
    "val_R2": r2_score(y_train, val_predictions),
    "val_r": pearsonr(val_predictions, y_train)[0],
    "val_r2": pearsonr(val_predictions, y_train)[0] ** 2,
    "train_R2": r2_score(y_train, train_predictions),
    "train_r": pearsonr(train_predictions, y_train)[0],
    "train_r2": pearsonr(train_predictions, y_train)[0] ** 2,
    "test_R2": r2_score(y_test, test_predictions),
    "test_r": pearsonr(test_predictions, y_test)[0],
    "test_r2": pearsonr(test_predictions, y_test)[0] ** 2,
    "demean_cv_R2": r2_score(
        train_split.demean_log_yield, train_split.demean_oos_prediction
    ),
    "demean_cv_r": pearsonr(
        train_split.demean_log_yield, train_split.demean_oos_prediction
    )[0],
    "demean_cv_r2": pearsonr(
        train_split.demean_log_yield, train_split.demean_oos_prediction
    )[0]
    ** 2,
    "demean_test_R2": r2_score(
        test_split.demean_log_yield, test_split.demean_oos_prediction
    ),
    "demean_test_r": pearsonr(
        test_split.demean_log_yield, test_split.demean_oos_prediction
    )[0],
    "demean_test_r2": pearsonr(
        test_split.demean_log_yield, test_split.demean_oos_prediction
    )[0]
    ** 2,
}
    # if return_oos_predictions:
    #     return d, oos_preds
    # else:
    #     return d

Begin with paramters:
F1: landsat-c2-l2_bands-r-g-b-nir-swir16-swir22_ZMB_20k-points_1024-features_yr-2009-2021_mn-4-9_lm-True_cm-True_wa-False_summary.feather
F2: sentinel-2-l2a_bands-2-3-4-8_ZMB_15k-points_1000-features_yr-2016-2022_mn-1-12_lm-False_cm-True_wa-False_summary.feather
One-hot encoding: True
Split: 0
Random state: 670487
N-splits: 5
Include climate: True
Climate vars: ndvi

Group indicies [0, 6144, 18144, 18156]
		  [6144, 18144, 18156, 18226]



In [50]:
fold_list = []
for i in range(n_splits):
    idx = len(list(kfold.split(y_train))[i][1])
    fold = np.repeat(i + 1, idx).tolist()
    fold_list.append(fold)
fold_list = np.array(fold_list).flatten().tolist()

In [66]:
#########################################     DE-MEAN TRAIN R2    #########################################
train_split = pd.DataFrame(
    np.repeat("train", len(x_train)), columns=["data_fold"], index=x_train.index
)
train_split = train_split.join(crop_yield[crop_yield.index.isin(x_train.index)])
train_split["oos_prediction"] = np.maximum(val_predictions, 0)
train_split["val_fold"] = fold_list
train_split = demean_by_group(
    train_split, predicted="oos_prediction", group=["district", "val_fold"]
)
r2_score(train_split.demean_log_yield, train_split.demean_oos_prediction)

0.218272560651123

In [67]:
#########################################     DE-MEAN TEST R2    #########################################
test_split = pd.DataFrame(
    {"data_fold": np.repeat("test", len(x_test))}, index=x_test.index
)
test_split = test_split.join(crop_yield[crop_yield.index.isin(x_test.index)])
test_split["oos_prediction"] = np.maximum(best_model.predict(x_test), 0)
test_split = demean_by_group(test_split, predicted="oos_prediction", group=["district"])
r2_score(test_split.demean_log_yield, test_split.demean_oos_prediction)

0.36670249579546255

In [ ]:

#########################################     OUT OF SAMPLE PREDICTIONS    #########################################
# oos_preds = pd.concat([train_split, test_split])
# oos_preds[["split", "random_state"]] = split, random_state

#########################################     SAVE RESULTS    #########################################
d = {
    "split": split,
    "random_state": random_state,
    "country": country_code[0],
    "year_range": yrs,
    "satellite_1": satellite1[0],
    "bands_1": bands1,
    "num_features_1": num_features1,
    "points_1": points1,
    "month_range_1": mns1,
    "limit_months_1": limit_months1,
    "crop_mask_1": crop_mask1,
    "weighted_avg_1": weighted_avg1,
    "satellite_2": satellite2[0],
    "bands_2": bands2,
    "num_features_2": num_features2,
    "points_2": points2,
    "month_range_2": mns2,
    "limit_months_2": limit_months2,
    "crop_mask_2": crop_mask2,
    "weighted_avg_2": weighted_avg2,
    "hot_encode": he,
    "total_n": len(x_all),
    "train_n": len(x_train),
    "test_n": len(x_test),
    "best_reg_param": [best_lambdas],
    "mean_of_val_R2": [best_scores],
    "val_R2": r2_score(y_train, val_predictions),
    "val_r": pearsonr(val_predictions, y_train)[0],
    "val_r2": pearsonr(val_predictions, y_train)[0] ** 2,
    "train_R2": r2_score(y_train, train_predictions),
    "train_r": pearsonr(train_predictions, y_train)[0],
    "train_r2": pearsonr(train_predictions, y_train)[0] ** 2,
    "test_R2": r2_score(y_test, test_predictions),
    "test_r": pearsonr(test_predictions, y_test)[0],
    "test_r2": pearsonr(test_predictions, y_test)[0] ** 2,
    "demean_cv_R2": r2_score(
        train_split.demean_log_yield, train_split.demean_oos_prediction
    ),
    "demean_cv_r": pearsonr(
        train_split.demean_log_yield, train_split.demean_oos_prediction
    )[0],
    "demean_cv_r2": pearsonr(
        train_split.demean_log_yield, train_split.demean_oos_prediction
    )[0]
    ** 2,
    "demean_test_R2": r2_score(
        test_split.demean_log_yield, test_split.demean_oos_prediction
    ),
    "demean_test_r": pearsonr(
        test_split.demean_log_yield, test_split.demean_oos_prediction
    )[0],
    "demean_test_r2": pearsonr(
        test_split.demean_log_yield, test_split.demean_oos_prediction
    )[0]
    ** 2,
}